# 0. Workflow:

1. Problem description, data characteristics, visualisation, distributions, etc.

2. Preprocessing:
   - missing data (delete, leave, imputation + explaination);
   - normalization/standarization (type of transformation + description, transformation categorical data);
   - separation - test and training sets (training/test, cross-validation, proportions of subsets);

3. Building and training neural network:
    - layers, neurons quantity; 
    - activation functions (+description, last layer should transform to binary output);
    - optimalisation method (+description, which one returns best results);
    - loss function and metrics (+description);
    - model fitting (epochs number, batch size, model evaluation (training result));
    - test set based classification (evaluation methods of binary classifier, including data type (should include just and only most desriptive metrics

4. Additional methods not presented in labs/lectures.

5. Present different parameters values, to decide, which ones caused accuracy and classification improvement.

In [ ]:
import yellowbrick
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
import keras
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras import optimizers
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
from keras import metrics as k_metrics
from sklearn import metrics as s_metrics

# 1. Problem description

...

# 2. Reading data using Pandas

In [ ]:
data = pd.read_csv("logreg.txt", sep = ";", dtype={'genotype': np.object})

# 3. Data characteristics

In [ ]:
# data.head()
# data.shape
# print(data.dtypes)

# data.BEFORE1.value_counts()
# data.BEFORE2.value_counts()
# data.BEFORE3.value_counts()

# data.BEHIND1.value_counts()
# data.BEHIND2.value_counts()
# data.BEHIND3.value_counts()

# descriptions_dataframe = data.describe(include='all')
# nulls = data.isnull().sum().to_frame(name='NA').T 
# summary_of_nulls = pd.concat([descriptions_dataframe, nulls])
# summary_of_nulls

# data.iloc[:,6:].apply(pd.value_counts)

# 4. Data visualisation

In [ ]:
# dims = (15.2, 10.75)
# fig, ax = plt.subplots(figsize=dims)

# bpl = sns.boxplot(
#     data=data,
#     x='genotype',
#     y='CALL',
# )

# bpl.figure.savefig(r"C:\Users\Acer\Desktop\Studia\Deep learning\project\SNP-predictions\boxplots\boxplot_genotype_CALL.png")


# 5. Preprocessing

## 5.1. Creating triplets - we can't analyse categorical data (nucleotides: A, T, G, C, N), so we must find numerical representation.

In [ ]:
tmp = pd.DataFrame()
tmp['BEFORE'] = data.BEFORE1 + data.BEFORE2 + data.BEFORE3
tmp['BEHIND'] = data.BEHIND1 + data.BEHIND2 + data.BEHIND3
data = data.drop(columns=['BEFORE1', 'BEFORE2', 'BEFORE3', 'BEHIND1', 'BEHIND2', 'BEHIND3'])

tmp = pd.get_dummies(tmp)
data = pd.concat([data, tmp], axis=1)

### 5.2. Histograms to check distributions

In [ ]:
# dims = (15.2, 10.75)
# fig, ax = plt.subplots(figsize=dims)
# sns_plot = sns.distplot(data['GQ'], ax = ax, hist=True, kde=True, 
#              bins=int(180/5), color = 'darkblue', 
#              hist_kws={'edgecolor':'black'},
#              kde_kws={'linewidth': 4})

# sns_plot.figure.savefig(r"C:\Users\Acer\Desktop\Programs\Python\Deep learning\project\SNP-predictions\Distribution plots\dist_GQ.png")


## 5.3. Split - training and test set

In [ ]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## 5.4. Missing data handling - imputation using kNN algorithm, Iterative Imputer, Simple Imputer

### KNN Imputer - hardware issues

In [ ]:
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=5, weights="uniform")
# imputer.fit_transform(X_train_to_impute)

### Iterative Imputer

In [ ]:
# imp = IterativeImputer(max_iter=10, random_state=0)
# imp.fit(X_train_to_impute)
# IterativeImputer(random_state=0)
# Iterative_DataFrame = pd.DataFrame(imp.transform(X_train_to_impute))

### Simple Imputer - mean

In [ ]:
imp_mean = SimpleImputer(strategy='mean')
imp_mean.fit(X_train)
X_train = pd.DataFrame(imp_mean.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imp_mean.transform(X_test), columns=X_train.columns)

### Simple Imputer - median

In [ ]:
# imp_median = SimpleImputer(strategy='median')
# imp_median.fit(X_train_to_impute)
# Median_DataFrame = pd.DataFrame(imp_median.transform(X_train_to_impute))

## 6. Standarization - change the values so that the distribution standard deviation from the mean equals one.

## 6.1. Standarize - MinMaxScaler

In [ ]:
# m_scaler = MinMaxScaler()
# m_scaler.fit(Iterative_DataFrame)
# m_scaler.transform(Iterative_DataFrame)
# X_train_Iterative = pd.DataFrame(m_scaler.transform(Iterative_DataFrame))

m_scaler = MinMaxScaler()
m_scaler.fit(X_train)
m_scaler.transform(X_train)
X_train = pd.DataFrame(m_scaler.transform(X_train), columns=X_train.columns)

m_scaler = MinMaxScaler()
m_scaler.fit(X_test)
m_scaler.transform(X_test)
X_test = pd.DataFrame(m_scaler.transform(X_test), columns=X_train.columns)

# m_scaler = MinMaxScaler()
# m_scaler.fit(Median_DataFrame)
# X_train_Median = pd.DataFrame(m_scaler.transform(Median_DataFrame))

# print(X_train_Iterative.shape)
# print(X_train_Median.shape)

# 7. Building neural network

In [ ]:
model = Sequential()

model.add(Dense(20, input_shape = (135,), activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(80, activation = 'relu'))
model.add(Dense(60, activation = 'relu'))
model.add(Dense(40, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(4, activation = 'relu'))
model.add(Dense(2, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[k_metrics.mae,
                      ])

X_val = X_train[:10000]
y_val = y_train[:10000]

history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=512,
                    validation_data=(X_val, y_val))

# 8. Plotting loss and mae change over epochs

In [ ]:
history_dict = history.history
print(history_dict.keys())

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

# dims = (15.2, 10.75)
fig = plt.figure(figsize=(15.2, 10.75))

plt.subplot(211)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# plt.legend()
# plt.show()

# plt.clf()
# # print(history_dict.keys())
mae = history_dict['mean_absolute_error']
val_mae = history_dict['val_mean_absolute_error']

plt.subplot(212)
plt.plot(epochs, mae, 'bo', label='Training mean absolute error')
plt.plot(epochs, mae, 'b', label='Validation mean absolute error')

plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')

plt.legend()
plt.show()

plt.savefig(r"C:\Users\Acer\Desktop\change.png")

results = model.evaluate(X_test, y_test)
print(results)

# 9. Function for predicting classes

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve

def predict_cls(arr, threshold):
    """
    Predicting classes based on given probability threshold
    """
    predicted = [0 if probability < threshold else 1 for probability in arr]
    return predicted

y_test = y_test.astype(int).to_numpy()

# 10. Confusion matrix for 0.5 threshold

In [ ]:
from sklearn import metrics

my_y_pred = predict_cls(model.predict(X_test), 0.5)
y_pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, my_y_pred)
print(conf_mat)

tp = conf_mat[0][0]
tn = conf_mat[1][1]
fp = conf_mat[0][1]
fn = conf_mat[1][0]

TPR_sensitivity = tp / (tp + fn)
SPC_specificity = tn / (fp + tn)
FPR = fp / (fp + tn)
FDR = fp / (fp + tp)

print("SENSITIVITY:", TPR_sensitivity)
print("SPECIFICITY:", SPC_specificity)
print("FPR:", FPR)
print("FDR:", FDR)
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)

# ROC, AUC

In [ ]:
plt.figure(figsize=(15.2, 10.75))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()